# Laboratorio 5

# Análisis de Sentimientos

### Cristina Bautista 161260
### Andy Castillo 18040
### Marco Fuentes 18188
### Abril Palencia 18198

In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import FreqDist
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cristinabautista/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cristinabautista/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from wordcloud import WordCloud, STOPWORDS

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
%matplotlib inline

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('GrammarandProductReviews.csv')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71044 entries, 0 to 71043
Data columns (total 25 columns):
id                      71044 non-null object
brand                   71044 non-null object
categories              71044 non-null object
dateAdded               71044 non-null object
dateUpdated             71044 non-null object
ean                     39065 non-null object
keys                    71044 non-null object
manufacturer            70903 non-null object
manufacturerNumber      70841 non-null object
name                    71044 non-null object
reviews.date            70977 non-null object
reviews.dateAdded       71044 non-null object
reviews.dateSeen        71044 non-null object
reviews.didPurchase     32158 non-null object
reviews.doRecommend     60429 non-null object
reviews.id              32158 non-null float64
reviews.numHelpful      32508 non-null float64
reviews.rating          71044 non-null int64
reviews.sourceURLs      71044 non-null object
reviews.text    

In [7]:
data = data[['brand','categories','manufacturer', 'name', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.numHelpful', 'reviews.rating', 'reviews.text', 'reviews.title', 'reviews.username']]

In [8]:
data.head()

,brand,categories,manufacturer,name,reviews.didPurchase,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.text,reviews.title,reviews.username
0,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),NaN,NaN,0.0,5,i love this album. it's very good. more to the...,Just Awesome,Joshua
1,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,True,NaN,NaN,5,Good flavor. This review was collected as part...,Good,Dorothy W
2,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,True,NaN,NaN,5,Good flavor.,Good,Dorothy W
3,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,False,False,NaN,1,I read through the reviews on here before look...,Disappointed,Rebecca
4,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,False,False,NaN,1,My husband bought this gel for us. The gel cau...,Irritation,Walker557


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71044 entries, 0 to 71043
Data columns (total 11 columns):
brand                  71044 non-null object
categories             71044 non-null object
manufacturer           70903 non-null object
name                   71044 non-null object
reviews.didPurchase    32158 non-null object
reviews.doRecommend    60429 non-null object
reviews.numHelpful     32508 non-null float64
reviews.rating         71044 non-null int64
reviews.text           71008 non-null object
reviews.title          70568 non-null object
reviews.username       70948 non-null object
dtypes: float64(1), int64(1), object(9)
memory usage: 6.0+ MB


In [10]:
data['brand'] = data['brand'].str.lower()
data['categories'] = data['categories'].str.lower()
data['manufacturer'] = data['manufacturer'].str.lower()
data['name'] = data['name'].str.lower()
data['reviews.text'] = data['reviews.text'].str.lower()
data['reviews.title'] = data['reviews.title'].str.lower()
data['reviews.username'] = data['reviews.username'].str.lower()

In [11]:
# Eliminar url
data['brand'] = data['brand'].str.replace(r'http\S+', ' ')
data['categories'] = data['categories'].str.replace(r'http\S+', ' ')
data['manufacturer'] = data['manufacturer'].str.replace(r'http\S+', ' ')
data['name'] = data['name'].str.replace(r'http\S+', ' ')
data['reviews.text'] = data['reviews.text'].str.replace(r'http\S+', ' ')
data['reviews.title'] = data['reviews.title'].str.replace(r'http\S+', ' ')
data['reviews.username'] = data['reviews.username'].str.replace(r'http\S+', ' ')

In [12]:
# Se elimino todo lo que no sean letras, como emojis, signos de puntuacion y numeros
data['brand'] = data['brand'].str.replace(r'[^A-Za-z\s]+', ' ')
data['categories'] = data['categories'].str.replace(r'[^A-Za-z]+', ' ')
data['manufacturer'] = data['manufacturer'].str.replace(r'[^A-Za-z]+', ' ')
data['name'] = data['name'].str.replace(r'[^A-Za-z\s]+', ' ')
data['reviews.text'] = data['reviews.text'].str.replace(r'[^A-Za-z]+', ' ')
data['reviews.title'] = data['reviews.title'].str.replace(r'[^A-Za-z]+', ' ')
data['reviews.username'] = data['reviews.username'].str.replace(r'[^A-Za-z]+', ' ')

In [13]:
# Se revisa que no tengan caracteres especiales las columnas deseadas
data.head()

,brand,categories,manufacturer,name,reviews.didPurchase,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.text,reviews.title,reviews.username
0,universal music,movies music books music r b movies tv movie b...,universal music group cash money,pink friday roman reloaded re up w dvd,NaN,NaN,0.0,5,i love this album it s very good more to the h...,just awesome,joshua
1,lundberg,food packaged foods snacks crackers snacks coo...,lundberg,lundberg organic cinnamon toast rice cakes,True,NaN,NaN,5,good flavor this review was collected as part ...,good,dorothy w
2,lundberg,food packaged foods snacks crackers snacks coo...,lundberg,lundberg organic cinnamon toast rice cakes,True,NaN,NaN,5,good flavor,good,dorothy w
3,k y,personal care medicine cabinet lubricant sperm...,k y,k y love sensuality pleasure gel,False,False,NaN,1,i read through the reviews on here before look...,disappointed,rebecca
4,k y,personal care medicine cabinet lubricant sperm...,k y,k y love sensuality pleasure gel,False,False,NaN,1,my husband bought this gel for us the gel caus...,irritation,walker


In [14]:
# Tokenize
tokenizer = RegexpTokenizer(r'\w+')

data['brand'] = data['brand'].apply(lambda x: tokenizer.tokenize(x))
data['categories'] = data['categories'].apply(lambda x: tokenizer.tokenize(x))
data['manufacturer'] = data['manufacturer'].apply(lambda x: tokenizer.tokenize(str(x)))
data['name'] = data['name'].apply(lambda x: tokenizer.tokenize(x))
data['reviews.text'] = data['reviews.text'].apply(lambda x: tokenizer.tokenize(str(x)))
data['reviews.title'] = data['reviews.title'].apply(lambda x: tokenizer.tokenize(str(x)))
data['reviews.username'] = data['reviews.username'].apply(lambda x: tokenizer.tokenize(str(x)))

In [15]:
# Stopwords
stopwords = stopwords.words('english')

In [16]:
data['reviews.text'] = data['reviews.text'].apply(lambda y: [w for w in y if w not in stopwords])
data['reviews.title'] = data['reviews.title'].apply(lambda y: [w for w in y if w not in stopwords])

In [17]:
# Se revisa que en reviews text y title no contengan stopwords
data.head()

,brand,categories,manufacturer,name,reviews.didPurchase,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.text,reviews.title,reviews.username
0,"[universal, music]","[movies, music, books, music, r, b, movies, tv...","[universal, music, group, cash, money]","[pink, friday, roman, reloaded, re, up, w, dvd]",NaN,NaN,0.0,5,"[love, album, good, hip, hop, side, current, p...",[awesome],[joshua]
1,[lundberg],"[food, packaged, foods, snacks, crackers, snac...",[lundberg],"[lundberg, organic, cinnamon, toast, rice, cakes]",True,NaN,NaN,5,"[good, flavor, review, collected, part, promot...",[good],"[dorothy, w]"
2,[lundberg],"[food, packaged, foods, snacks, crackers, snac...",[lundberg],"[lundberg, organic, cinnamon, toast, rice, cakes]",True,NaN,NaN,5,"[good, flavor]",[good],"[dorothy, w]"
3,"[k, y]","[personal, care, medicine, cabinet, lubricant,...","[k, y]","[k, y, love, sensuality, pleasure, gel]",False,False,NaN,1,"[read, reviews, looking, buying, one, couples,...",[disappointed],[rebecca]
4,"[k, y]","[personal, care, medicine, cabinet, lubricant,...","[k, y]","[k, y, love, sensuality, pleasure, gel]",False,False,NaN,1,"[husband, bought, gel, us, gel, caused, irrita...",[irritation],[walker]
